In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import numpy as np

https://www.kaggle.com/datasets/ayeshasiddiqa123/customer-shopping-behavior-dataset

In [2]:
df = pd.read_csv("shopping_behavior_updated.csv")
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [4]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.749949,25.351538
std,1125.977353,15.207589,23.685392,0.716223,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.700000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [5]:
df.isna().sum()

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

In [6]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [7]:
labels = ['Young Adult', 'Adult', 'Middle-aged', 'Senior']
df['age_group'] = pd.qcut(df['age'], q=4, labels = labels)

In [8]:
df[['age','age_group']].head(5)

,age,age_group
0,55,Middle-aged
1,19,Young Adult
2,50,Middle-aged
3,21,Young Adult
4,45,Middle-aged


In [9]:
frequency_mapping = {
    'Fortnightly': 14,
    'Weekly': 7,
    'Monthly': 30,
    'Quarterly': 90,
    'Bi-Weekly': 14,
    'Annually': 365,
    'Every 3 Months': 90
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)

In [10]:
df[['purchase_frequency_days','frequency_of_purchases']].head(5)

,purchase_frequency_days,frequency_of_purchases
0,14,Fortnightly
1,14,Fortnightly
2,7,Weekly
3,7,Weekly
4,365,Annually


In [11]:
(df['discount_applied'] == df['promo_code_used']).all()

np.True_

In [12]:
df = df.drop('promo_code_used', axis=1)

In [13]:
df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_group,purchase_frequency_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,14,Venmo,Fortnightly,Middle-aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23,Credit Card,Weekly,Middle-aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,31,PayPal,Annually,Middle-aged,365


In [16]:
%pip install sqlalchemy psycopg2-binary

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 20.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 31.4 MB/s  0:00:00

   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/

In [17]:
from sqlalchemy import create_engine

# Step 1: Connect to PostgreSQL
# Replace placeholders with your actual details
username = "postgres"      # default user
password = "ettmt123" # the password you set during installation
host = "localhost"         # if running locally
port = "5432"              # default PostgreSQL port
database = "customer_behavior"    # the database you created in pgAdmin

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# Step 2: Load DataFrame into PostgreSQL
table_name = "customer"   # choose any table name
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'customer' in database 'customer_behavior'.


In [ ]:
# X = df.drop(columns=[
#     "customer_id",
#     "age_group",
#     "purchase_frequency_days",
#     "frequency_of_purchases"
# ])

# y = df["frequency_of_purchases"]

# binary_cols = ["subscription_status", "discount_applied"]

# X[binary_cols] = X[binary_cols].replace({"Yes": 1, "No": 0})

# X = X.drop(columns=["item_purchased", "color", "location"])

C:\Users\tomin\AppData\Local\Temp\ipykernel_8724\3043370507.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[binary_cols] = X[binary_cols].replace({"Yes": 1, "No": 0})


In [ ]:
# cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
# num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
# print("Numeric:", num_cols)
# print("Categorical:", cat_cols)

Numeric: ['age', 'purchase_amount', 'review_rating', 'subscription_status', 'discount_applied', 'previous_purchases']
Categorical: ['gender', 'category', 'size', 'season', 'shipping_type', 'payment_method']


In [ ]:
# preprocess = ColumnTransformer(
#     transformers=[
#         ("num", StandardScaler(), num_cols),
#         ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
#     ]
# )

# processed_data = preprocess.fit_transform(X)
# processed_data.shape

(3900, 32)

In [ ]:
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# def cv_classification_metrics(model, X, y, cv):
#     acc = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
#     f1  = cross_val_score(model, X, y, cv=cv, scoring="f1_macro")
    
#     return pd.DataFrame({
#         "Accuracy": acc,
#         "F1_macro": f1
#     }).agg(["mean", "std"]).T

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# dt_pipe = Pipeline([
#     ("preprocess", preprocess),
#     ("model", DecisionTreeClassifier(
#         max_depth=10,
#         random_state=42
#     ))
# ])

# rf_pipe = Pipeline([
#     ("preprocess", preprocess),
#     ("model", RandomForestClassifier(
#         n_estimators=300,
#         max_depth=12,
#         random_state=42,
#         n_jobs=-1
#     ))
# ])

In [ ]:
# dt_scores = cv_classification_metrics(dt_pipe, X, y, kfold)
# rf_scores = cv_classification_metrics(rf_pipe, X, y, kfold)

# pd.concat(
#     {"DecisionTree": dt_scores, "RandomForest": rf_scores},
#     axis=1
# )

DecisionTree           RandomForest          
                 mean       std         mean       std
Accuracy     0.141026  0.014979     0.139487  0.010111
F1_macro     0.137579  0.015837     0.136786  0.010308

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, stratify=y, random_state=42
# )

# rf_pipe.fit(X_train, y_train)

# y_pred = rf_pipe.predict(X_test)

In [ ]:
# print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

      Annually       0.17      0.18      0.17       114
     Bi-Weekly       0.11      0.11      0.11       109
Every 3 Months       0.09      0.09      0.09       117
   Fortnightly       0.12      0.10      0.11       108
       Monthly       0.15      0.15      0.15       111
     Quarterly       0.12      0.14      0.13       113
        Weekly       0.07      0.06      0.07       108

      accuracy                           0.12       780
     macro avg       0.12      0.12      0.12       780
  weighted avg       0.12      0.12      0.12       780



In [ ]:
# confusion_matrix(y_test, y_pred)

array([[20, 18, 20, 17, 15, 10, 14],
       [13, 12, 13,  9, 14, 24, 24],
       [13, 24, 11, 15, 16, 24, 14],
       [15, 12, 26, 11, 17, 15, 12],
       [12, 15, 19, 16, 17, 21, 11],
       [14, 20, 15, 13, 21, 16, 14],
       [28, 12, 15,  9, 17, 20,  7]])